<a href="https://colab.research.google.com/github/GiulioAppetito/Progetto_MOBD_2122/blob/main/training_routine3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [365]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [366]:
%cd /content/drive/My\ Drive/ProgettoMOBD

/content/drive/My Drive/ProgettoMOBD


In [367]:
import sklearn
import pandas as pd
import numpy as np

from numpy import array
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.experimental import enable_iterative_imputer
from sklearn_pandas import CategoricalImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import KMeansSMOTE
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn import tree

#serialization
from joblib import dump
from sklearn.pipeline import Pipeline
from myencoder import MyEncoder
import pickle

#graphs
import graphviz
import matplotlib.pyplot as plt

#evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

np.random.seed = 123457

In [ ]:
# LETTURA DATASET
data = pd.read_csv("train.csv", sep=",")
print(data.head())

In [ ]:
data['F13'] = data['F13'].replace(['USA','Jamaica','Caribbean','Cuba','Dominican-Republic','Guatemala','Haiti','Honduras','Nicaragua','El-Salvador','Puerto-Rico','Mexico','Canada'],'America N')
data['F13'] = data['F13'].replace(['South','Columbia','Ecuador','Peru'],'America S')
data['F13'] = data['F13'].replace(['Cambodia','China','Hong','India','Iran','Japan','Philippines','Taiwan','Vietnam','Thailand','Laos'],'Asia')
data['F13'] = data['F13'].replace(['England','France','Germany','Greece','Hungary','Ireland','Italy','Netherlands','Poland','Portugal','Scotland','Yugoslavia'],'Europe')

print(data.head())

In [370]:
# print("\nLabels: {}".format(np.unique(data.Y, return_counts=True)))
# print("\n# of NaNs values for each column:")
# print(data.isnull().sum(axis=0))

# dropping nan values: lowers the performance of circa 10%
# data = data.dropna()

x = data.iloc[:, :-1].to_numpy()
y = data.iloc[:, -1].to_numpy()
# print(x.shape)
# print(y.shape)

# stratify: to mantain the same mean as y for y_tr and y_ts
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.24, random_state=258, stratify=y)

# print(x_tr.shape)
# print(y_tr.shape)
# print(x_ts.shape)
# print(y_ts.shape)

#print(np.unique(y, return_counts=True)[1]/len(y))
#print(np.unique(y_tr, return_counts=True)[1]/len(y_tr))
#print(np.unique(y_ts, return_counts=True)[1]/len(y_ts))

In [372]:
# sostituzione valori nan: dato che sono presenti valori categorici, e nello specifico sono proprio
# tre colonne di quel tipo ad avere valori mancanti, abbiamo optato per un semplice Imputer 
# che sostituisse con i più frequenti.

imputer = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
x_tr_notnan = imputer.fit_transform(x_tr)
# la sostituzione sul test set avviene con la media/mediana/most_frequent calcolati sul training set
x_ts_notnan = imputer.transform(x_ts)

In [373]:
# dato che però a seguito della sostituzione dei valori nan abbiamo ndarrays, 
# dobbiamo riportarci a dataframes per applicare la one hot coded technique

x_tr_df = pd.DataFrame(x_tr_notnan)
x_ts_df = pd.DataFrame(x_ts_notnan)

x_tr_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']
x_ts_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']

pd.set_option('display.max_columns',None)

# one hot code: stiamo rimpiazzando ogni colonna corrispondente ad un attributo categorico
# con un set di colonne della cardinalità del range dell'attributo in questione, 
# a ciascuna delle quali corrisponde appunto un elemento nel dominio dell'attributo, ed ogni tupla
# del set avrà, corrispondentemente a queste nuove colonne inserite, valore 1 su quella
# rispettiva al valore che si ha nel set originale, e zero per tutte le altre
# esempio: 
#          ..| Fi | ..  ---> ..| K1 | .. | Ki | .. | Kn |..
#          ..| Ki | ..  ---> ..| 0  | .. | 1  | .. | 0  |..

# ripetiamo questa operazione sia sul training set che sul test set

encoder = MyEncoder(np.nan)
encoded_df_tr = encoder.fit()
print(encoded_df_tr)

encoded_df_ts = encoder.transform()

This object is only an encoder
None
This object is only an encoder


In [ ]:
# SCALING
print("Robust Scaler")
scaler = RobustScaler()
x_tr_scaled = scaler.fit_transform(encoded_df_tr)
x_ts_scaled = scaler.transform(encoded_df_ts)

print("Media distribuzione di partenza: {}".format(np.mean(encoded_df_tr, axis=0)))
print("Media distribuzione scalata: {}".format(np.mean(x_tr_scaled, axis=0)))

print("Deviazione standard distribuzione di partenza: {}".format(np.std(encoded_df_tr, axis=0)))
print("Deviazione standard distribuzione scalata: {}".format(np.std(x_tr_scaled, axis=0)))

print("Media distribuzione di partenza: {}".format(np.mean(encoded_df_ts, axis=0)))
print("Media distribuzione scalata: {}".format(np.mean(x_ts_scaled, axis=0)))

print("Deviazione standard distribuzione di partenza: {}".format(np.std(encoded_df_ts, axis=0)))
print("Deviazione standard distribuzione scalata: {}".format(np.std(x_ts_scaled, axis=0)))

In [ ]:
# SERIALIZZAZIONE PIPELINE PER LA ROUTINE DI TEST
import pickle
import dill
import cloudpickle

preprocessor = Pipeline(steps=[('imputer',imputer),('encoder',encoder),('scaler',scaler)])
#dump(preprocessor, 'preprocessor.joblib')
with open('preprocessor.pkl', 'wb') as out_file:
    dill.dump(preprocessor, out_file)


In [ ]:
# BALANCING: oversampling

print(x_tr_scaled.shape)

balancer = KMeansSMOTE(sampling_strategy=1, k_neighbors=5)
x_tr_balanced, y_tr_balanced = balancer.fit_resample(x_tr_scaled, y_tr)

print(x_tr_balanced.shape)

In [ ]:
# ANOMALY DETECTION CON ISOLATION FOREST
print("Initial distribution")
print(len(x_tr))
print(np.unique(y_tr, return_counts=True)[1]/len(y_tr))

anomaly_detector = IsolationForest(n_estimators = 150, bootstrap = True, random_state = 42)
anomaly_detector.fit(x_tr_balanced)
is_inlier = anomaly_detector.predict(x_tr_balanced)

print(np.unique(is_inlier, return_counts=True))
x_tr_not_anomalous = x_tr_balanced[is_inlier==1,:]
y_tr_not_anomalous = y_tr_balanced[is_inlier==1]

print("Distribution after anomaly detection")
print(len(x_tr_not_anomalous))
print(np.unique(y_tr_not_anomalous, return_counts=True)[1]/len(y_tr_not_anomalous))

In [ ]:
# BALANCING: undersampling

#questa tecnica non ha fornito i risultati sperati

n_classes = len(np.unique(y_tr_not_anomalous))
fig, ax = plt.subplots()
sizes = [len(y_tr_not_anomalous[y_tr_not_anomalous==i]) for i in range(n_classes)]
class_names = [chr(ord('A')+i) for i in range(n_classes)]

ax.pie(sizes, labels=class_names, autopct='%1.1f%%',shadow=True,startangle=90)
ax.axis("equal")
plt.show()

sampler = RandomUnderSampler(sampling_strategy=1)
x_tr_balanced, y_tr_balanced = sampler.fit_resample(x_tr_not_anomalous, y_tr_not_anomalous)

print("Size del dataset prima del sampler: {}".format(x_tr_not_anomalous.shape[0]))
print("Size del dataset dopo il sampler: {}".format(x_tr_balanced.shape[0]))

fig, ax = plt.subplots()
sizes = [len(y_tr_balanced[y_tr_balanced==i]) for i in range(n_classes)]
class_names = [chr(ord('A')+i) for i in range(n_classes)]

ax.pie(sizes, labels=class_names, autopct='%1.1f%%',shadow=True,startangle=90)
ax.axis("equal")
plt.show()

In [ ]:
def train_evaluate(model, x_tr, y_tr, x_ts, y_ts):
  model.fit(x_tr, y_tr)
  y_tr_pred = model.predict(x_tr)
  y_ts_pred = model.predict(x_ts)
  print("Accuracy on training set {}".format(accuracy_score(y_tr, y_tr_pred)))
  print("F1 score on training set {}".format(f1_score(y_tr, y_tr_pred)))
  print(confusion_matrix(y_tr, y_tr_pred))
  print("Accuracy on test set {}".format(accuracy_score(y_ts, y_ts_pred)))
  print("F1 score on test set {}".format(f1_score(y_ts, y_ts_pred)))
  print(confusion_matrix(y_ts, y_ts_pred))

In [ ]:
# CLASSIFICATION: METODI ENSEMBLE
#print("******* RandomForestClassifier")
#classifier = RandomForestClassifier(max_depth=5, n_estimators=100)
#train_evaluate(classifier, x_tr_not_anomalous, y_tr_not_anomalous, x_ts_scaled, y_ts)

# i classificatori che hanno permesso di ottenere le migliori performance
# sono stati senza dubbio gli alberi, e in particolare il gradient boosting classifier (per ora siamo a: 87,54-87,62-87,71)

print("******* GradientBoostingClassifier")
classifier = GradientBoostingClassifier(max_depth=5, n_estimators=220)
train_evaluate(classifier, x_tr_not_anomalous, y_tr_not_anomalous, x_ts_scaled, y_ts)

print("******* GradientBoostingClassifier")
classifier2 = GradientBoostingClassifier(max_depth=5, n_estimators=220)
train_evaluate(classifier2, x_tr_balanced, y_tr_balanced, x_ts_scaled, y_ts)

import lightgbm as lgb
from lightgbm import LGBMClassifier

print("******* LGBMClassifier")
classifier3 = LGBMClassifier(max_depth=5, n_estimators=220)
train_evaluate(classifier3, x_tr_balanced, y_tr_balanced, x_ts_scaled, y_ts)

******* GradientBoostingClassifier
Accuracy on training set 0.9317338741960449
F1 score on training set 0.9303794023188722
[[17821   886]
 [ 1672 17092]]
Accuracy on test set 0.8746001279590531
F1 score on test set 0.7219069239500567
[[5563  370]
 [ 610 1272]]
******* GradientBoostingClassifier
Accuracy on training set 0.9319512454758356
F1 score on training set 0.9304009363346852
[[17928   859]
 [ 1698 17091]]
Accuracy on test set 0.8766474728087013
F1 score on test set 0.7251995438996578
[[5579  354]
 [ 610 1272]]
******* LGBMClassifier
Accuracy on training set 0.9262295081967213
F1 score on training set 0.9243449781659387
[[17870   917]
 [ 1855 16934]]
Accuracy on test set 0.8747280870121561
F1 score on test set 0.7197251646149441
[[5579  354]
 [ 625 1257]]


In [ ]:
dump(classifier2, 'classifier.joblib') 

['classifier.joblib']